In [1]:
import matplotlib as plt 
import numpy as np 
import random
import pandas as pd
from collections import defaultdict
import random

In [2]:
def createList(r1, r2):
        if (r1 == r2):
            return r1
        else:
            res = []
            while(r1 != r2):
                res.append(r1)
                r1 += 1
                if (r1 > 50):
                    r1 = r1-50
            res.append(r2)
        return res 

In [3]:
class Graph():
    
    def __init__(self):
        
        #create graph
        self.graph = {}
        for i in range(1,51):
            if i-1 == 0:
                self.graph[i] = [2, 50]
            elif i+1 == 51:
                self.graph[i] = [1, 49]
            else:
                self.graph[i] = [i+1, i-1]
        
        #create list of nodes
        self.nodes = createList(1, 50)
                
    def addEdge(self):
        boo1 = True
        while boo1:
            if len(self.nodes) == 0:
                boo1 = False
                break
            val = random.choice(self.nodes)
            self.nodes.remove(val)
            valstart = val-5
            valend = val+5
            if (valstart < 1):
                valstart = valstart+50
            if (valend > 50):
                valend = valend-50
            val_nodes = createList(valstart, valend)
            if(val == 1):
                val_nodes.remove(50)
            elif(val == 50):
                val_nodes.remove(1)
            else:
                val_nodes.remove(val-1)
                val_nodes.remove(val+1)
            boo2 = False
            while len(val_nodes) > 0:
                newnode = random.choice(val_nodes)
                if newnode in self.nodes:
                    self.nodes.remove(newnode)
                    self.graph[val].append(newnode)
                    self.graph[newnode].append(val)
                    boo2 = True
                    break
                else:
                    val_nodes.remove(newnode)
                    
            if boo2:
                break
        if boo1 == False:
            return False
        else:
            return True

In [4]:
#start position, end position, backwards path to take
#returns path
def BFS(start, end, graph):
 
        # a queue of the vertices whose path is to be scanned
        # boolean array for checking if node has been visited at least once or not
        # all moves in path set to -1
        queue = []
        visited = [False for i in range(51)]
        back_path = [-1 for i in range(51)]
     
        # start is visited and added to the queue
        visited[start] = True
        queue.append(start)
  
        # BFS algorithm - look at all neighbors first
        while (len(queue) != 0):
            u = queue[0]
            queue.pop(0)
            for i in range(len(graph[u])):
                if (visited[graph[u][i]] == False):
                    visited[graph[u][i]] = True
                    back_path[graph[u][i]] = u
                    queue.append(graph[u][i])
  
                    # We stop BFS when we find
                    # destination.
                    if (graph[u][i] == end):
                        path = []
                        crawl = end
                        path.append(crawl)
                        
                        #reversing back_path and removing -1 from nodes that are not in the path to get full path
                        while (back_path[crawl] != -1):
                            path.insert(0, back_path[crawl])
                            crawl = back_path[crawl]
                        return path
  
        return [end]

In [5]:
class Predator():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure that predator doesn't start where the agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def pred_move(self, g, agentpos):
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths and moving to next neighbor
        new_path = random.randint(0,(len(shortest_path)-1))
        self.pos = shortest_path[new_path][0]
        return self

In [6]:
class Prey():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure prey doesn't start where agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def prey_move(self, g):
        prey_loc_neighbours = [self.pos]
        for i in g.graph[self.pos]:
            prey_loc_neighbours.append(i)
        #randomly choose neighbor to move to (or staying put)
        self.pos = random.choice(prey_loc_neighbours)
        return self

In [27]:
# returns fail - 0 if succeeds, 1 if fails by predator, 2 if fails by timeout
def Agent3(g):
    # starting positions of agent, predator, and prey (don't know where prey is)
    agent_pos = random.randint(1,50)
    predator = Predator(agent_pos)
    prey = Prey(agent_pos)
    fail = 0
    steps = 0
    # keep track of agent's path
    path = []
    path.append(agent_pos)
    
    # probability of prey location starts randomly anywhere except where the agent is (sort so highest prob is first)
    prey_prob = [0]*50
    for i in range(len(prey_prob)):
        prey_prob[i] = 1/49
    prey_prob[agent_pos-1] = 0
    
    while (fail == 0 and steps <= 100):
        
        print("Actual Positions: " + str(agent_pos) + ", " + str(predator.pos) + ", "+ str(prey.pos))
        
        # survey the node with the highest probability and see if the prey is there
        highest_prob = max(prey_prob)
        highest = []
        for i in range(len(prey_prob)):
            if(prey_prob[i] == highest_prob):
                highest.append((prey_prob[i], i+1))
        search_for_prey = random.choice(highest)
        
        # first update of prey_prob
        if(search_for_prey[1] == prey.pos): #everything = 0 but the position where the prey is found
            for prob in range(len(prey_prob)):
                prey_prob[prob] = 0
            prey_prob[prey.pos-1] = 1
        else:
            divisor = 1-highest_prob
            for i in range(len(prey_prob)): #all other prob are divided by 1-prob of surveyed location
                    prey_prob[i] = prey_prob[i]/divisor
            prey_prob[search_for_prey[1]-1] = 0 #prey is not where we surveyed
            
        print("Probabilities after 1st update:")
        for prob in range(len(prey_prob)):
            print(""+str(prob)+": "+ str(prey_prob[prob]))
        print("Sum of all probabilities: "+ str(sum(prey_prob)))
        
        # getting all possible next positions of agent 3
        neighbors = g.graph[agent_pos]
        # moves = list of tuples - each path has a tuple with the distance to predator and 
        # the neighbor they would move to for that path
        moves = []
        path_to_predator = []
        curr_to_pred = len(BFS(agent_pos, predator.pos, g.graph))
        
        for neighbor in neighbors:
            
            # shortest path from neighbor to probable prey and shortest path from neighbor to predator
            path_to_prey = BFS(neighbor, search_for_prey[1], g.graph)
            path_to_predator = BFS(neighbor, predator.pos, g.graph)
            
            #print("Prey and Predator Moves")
            #print(path_to_prey)
            #print(path_to_predator)
            
            # only adding tuple if we're not getting closer to the predator
            if (len(path_to_predator) >= curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves.append(tup)
        
        # if no move gets us not closer to the predator, we stay put
        if(len(moves) > 0):
            moves_sorted = sorted(moves, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            agent_pos = moves_sorted[0][2]
        path.append(agent_pos)
        
        if(prey.pos == agent_pos):
            break
        else:
            #second update of prey_prob
            divisor = 1-highest_prob
            for i in range(len(prey_prob)):
                    prey_prob[i] = prey_prob[i]/divisor
            prey_prob[search_for_prey[1]-1] = 0
        
        #move predator and prey
        predator.pred_move(g, agent_pos)
        prey.prey_move(g)
        
        #check if there is a success or fail
        if(predator.pos == agent_pos):
            fail = 1
            break
        elif(prey.pos == agent_pos):
            break
        steps += 1
        
        #third update of prey_prob
        temp_belief_vector = prey_prob.copy()
        for i in range(len(prey_prob)):
            mag_sum = 0
            i_neighbours = g.graph[i+1]
            for neighbour in i_neighbours:
                mag_sum += ((temp_belief_vector[neighbour-1])*(1/(len(i_neighbours)+1)))
            mag_sum += ((temp_belief_vector[agent_pos-1])*(1/(len(i_neighbours)+1)))
            prey_prob[i] = mag_sum
        
        print("Probabilities after last update:")
        for prob in range(len(prey_prob)):
            print(""+str(prob)+": "+ str(prey_prob[prob]))
        print("Sum of all probabilities: "+ str(sum(prey_prob)))
        
    #fail if timeout error    
    if(steps >= 100):
        fail = 2     
    return fail, path

In [28]:
wins = 0
losses = 0
timeout = 0
times = 0

while times < 1:
    
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()

    res, path = Agent3(g)
    
    if(res == 0):
        wins += 1
    elif(res == 1):
        losses += 1
    else:
        timeout += 1
    times += 1
    #print("Path is: ")
    #print(path)
    
print('Number of wins: ' + str(wins))
print('Number of losses: ' + str(losses))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))

Actual Positions: 9, 5, 18
Probabilities after 1st update:
0: 0.020833333333333332
1: 0.020833333333333332
2: 0.020833333333333332
3: 0.020833333333333332
4: 0.020833333333333332
5: 0.020833333333333332
6: 0.020833333333333332
7: 0.020833333333333332
8: 0.0
9: 0.020833333333333332
10: 0.020833333333333332
11: 0.020833333333333332
12: 0.020833333333333332
13: 0
14: 0.020833333333333332
15: 0.020833333333333332
16: 0.020833333333333332
17: 0.020833333333333332
18: 0.020833333333333332
19: 0.020833333333333332
20: 0.020833333333333332
21: 0.020833333333333332
22: 0.020833333333333332
23: 0.020833333333333332
24: 0.020833333333333332
25: 0.020833333333333332
26: 0.020833333333333332
27: 0.020833333333333332
28: 0.020833333333333332
29: 0.020833333333333332
30: 0.020833333333333332
31: 0.020833333333333332
32: 0.020833333333333332
33: 0.020833333333333332
34: 0.020833333333333332
35: 0.020833333333333332
36: 0.020833333333333332
37: 0.020833333333333332
38: 0.020833333333333332
39: 0.020833